In [ ]:
!pip3 install python-dotenv
!pip3 install llama-index
!pip3 install nebula2-python
!pip3 install langchain

In [2]:
from IPython.display import Markdown, display



In [3]:
# For OpenAI

import os
from dotenv import load_dotenv

hello


In [4]:
load_dotenv()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO) # logging.DEBUG for more verbose output

In [ ]:
os.environ['OPENAI_API_KEY']


In [ ]:
#!pip install llama_index
# !pip install langchain
# !pip install IPython

In [7]:


from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore


from langchain import OpenAI
from IPython.display import Markdown, display


# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-002"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
import os
import json
import openai
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index import LangchainEmbedding
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext
)

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

import logging
import sys

In [99]:
# Create a NebulaGraph cluster with:
    # Option 0 for machines with Docker: `curl -fsSL nebula-up.siwei.io/install.sh | bash`
    # Option 1 for Desktop: NebulaGraph Docker Extension https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext

# If not, create it with the following commands from NebulaGraph's console:
    # CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
    # :sleep 10;
    # USE llamaindex;
    # CREATE TAG entity(name string);
    # CREATE EDGE relationship(relationship string);
    # :sleep 10;
    # CREATE TAG INDEX entity_index ON entity(name(256));

%pip install ipython-ngql nebula3-python

os.environ['NEBULA_USER'] = "root"
os.environ['NEBULA_PASSWORD'] = "<password>" # default is "nebula
os.environ['NEBULA_ADDRESS'] = "0.0.0.0:9669" # assumed we have NebulaGraph installed locally

# space_name = "llamaindex"
space_name = "linkedin_v2"
edge_types, rel_prop_names = ["relationship"], ["relationship"] # default, could be omit if create from an empty kg
tags = ["entity"] # default, could be omit if create from an empty kg

Note: you may need to restart the kernel to use updated packages.


In [100]:
graph_store = NebulaGraphStore(space_name=space_name, edge_types=edge_types, rel_prop_names=rel_prop_names, tags=tags)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [18]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Guardians of the Galaxy Vol. 3'], auto_suggest=False)

# Kilian LinkedIn MVP

In [ ]:
# CREATE SPACE linkedin_v2(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
# :sleep 10;
# USE linkedin_v2;
# CREATE TAG entity(name string);
# CREATE EDGE relationship(relationship string);
# :sleep 10;
# CREATE TAG INDEX entity_index ON entity(name(256));

In [74]:
def read_md_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    return content

md_content = read_md_file('/home/kilian/google-drive/knowledgedb/231003/231003_linkedIn_posts.md')
print(md_content)


**To: Wey (Siwei) Gu & Lynn Bender**


Dear Lynn and Wey,

I hope this message finds you in good spirits. As recent additions to my LinkedIn network, I’ve admired your contributions and wondered if we could embark on an exciting collaboration.

**Motivation:**  
While preparing to share a recent article on RAM-based GNNs - Amazon Researchers Introduce DistTG, I realized its relevance to my previous posts. The current LinkedIn design, however, doesn't facilitate easy referencing of such connected content.

**Context:**  
LinkedIn, in its current state, encourages fast-paced, buzzword-heavy content, often neglecting the depth and interconnectedness of knowledge. Rather than building on previous insights or referencing established knowledge (standing on the shoulders of giants), content risks becoming ephemeral and redundant.

**Solution:**  
It's time for platforms like LinkedIn, and potentially others like Facebook, Instagram, or Twitter, to evolve. Imagine a version-controlled system, 

In [75]:
# Convert the custom text into the desired format
document2 = Document(id="custom_doc_1", text=md_content, metadata={"source": "custom"})

print(type(document2))  # This should output <class 'llama_index.schema.Document'>


<class 'llama_index.schema.Document'>


In [76]:
document2

Document(id_='8f160064-d6e2-4ccb-a0c6-6197e4dc7f24', embedding=None, metadata={'source': 'custom'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='3e5b48ab9c3c8d9eb3b2a475e3cebbcb4a0719d54ebf231b6ae37fcd83dbc755', text='**To: Wey (Siwei) Gu & Lynn Bender**\n\n\nDear Lynn and Wey,\n\nI hope this message finds you in good spirits. As recent additions to my LinkedIn network, I’ve admired your contributions and wondered if we could embark on an exciting collaboration.\n\n**Motivation:**  \nWhile preparing to share a recent article on RAM-based GNNs - Amazon Researchers Introduce DistTG, I realized its relevance to my previous posts. The current LinkedIn design, however, doesn\'t facilitate easy referencing of such connected content.\n\n**Context:**  \nLinkedIn, in its current state, encourages fast-paced, buzzword-heavy content, often neglecting the depth and interconnectedness of knowledge. Rather than building on previous insights or referencing e

In [77]:
kg_index = KnowledgeGraphIndex.from_documents(
    [document2],
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    service_context=service_context,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

(You, decided to share, conversation countries)
(You, decided to share, global open-source network)
(You, would be contributing, collective knowledge pool)
(You, would be exposing, myriad of question-answer universes)
(Gandhi, Be the change you want to see in the world)
(I, have honestly no idea, happens if more than one person decides to continue the conversation)
(Please feel free to be part, social experiment)
(Answer and share again, ripple effect)
(companies, have more than 1000 employees)
(companies, need a transformative approach)
(data lake, is, bedrock of data storage and management)
(ELT, is, robust digestion layer)
(feature-store, is, bridge for User Memory Retrieval)
(graph-DB, is, maps data relationships)
(Service-Mesh, ensures communication between, Developers & Data Engineers)
(Service-Mesh, ensures communication between, Scientists & Analysts)
(Service-Mesh, ensures communication between, machines)
(Service-Mesh, can consider, agent-pool-handler approach)
(Service-Mesh,

In [78]:
# install related packages, password is nebula by default
%pip install ipython-ngql networkx pyvis
%load_ext ngql
%ngql --address 0.0.0.0 --port 9669 --user root --password <password>

Note: you may need to restart the kernel to use updated packages.
Connection Pool Created
INFO:nebula3.logger:Get connection to ('0.0.0.0', 9669)


,Name
0,demo_data_lineage
1,linkedin_as_knowledge_graph
2,linkedin_v2
3,llamaindex


In [115]:
# Query some random Relationships with Cypher
# %ngql USE llamaindex;
%ngql USE linkedin_v2;
%ngql MATCH ()-[e]->() RETURN e LIMIT 100000

INFO:nebula3.logger:Get connection to ('0.0.0.0', 9669)
INFO:nebula3.logger:Get connection to ('0.0.0.0', 9669)


,e
0,"(""@alanbecker"")-[:relationship@131303000615340..."
1,"(""@prosperpython"")-[:relationship@131303000615..."
2,"(""AI composing"")-[:relationship@-8161707173856..."
3,"(""Aldi"")-[:relationship@-6959414471673446724{r..."
4,"(""Aldi"")-[:relationship@-6959414471673446724{r..."
...,...
158,"(""whisper api"")-[:relationship@201934314854289..."
159,"(""whisper api"")-[:relationship@201934314854289..."
160,"(""work"")-[:relationship@6158135741405254097{re..."
161,"(""x"")-[:relationship@-232941839542300431{relat..."


In [116]:
# draw the result

%ng_draw

<class 'pyvis.network.Network'> |N|=214 |E|=163

In [101]:
# llm=OpenAI(temperature=0, model_name="text-davinci-002")

In [111]:
from llama_index.query_engine import KnowledgeGraphQueryEngine

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

In [112]:
response = query_engine.query(
    "what do i say about math?",
)
display(Markdown(f"<b>{response}</b>"))

Graph Store Query:

MATCH (p:`person`)-[:directed]->(m:`movie`) WHERE m.`movie`.`name` == 'The Godfather'
RETURN p.`person`.`name`;
ERROR:llama_index.graph_stores.nebulagraph:Query failed. Query: MATCH (p:`person`)-[:directed]->(m:`movie`) WHERE m.`movie`.`name` == 'The Godfather'
RETURN p.`person`.`name`;, Param: {}Error message: Query failed. Query: MATCH (p:`person`)-[:directed]->(m:`movie`) WHERE m.`movie`.`name` == 'The Godfather'
RETURN p.`person`.`name`;, Param: {}Error message: SemanticError: `person': Unknown tag
ERROR:llama_index.graph_stores.nebulagraph:Query failed. Query: MATCH (p:`person`)-[:directed]->(m:`movie`) WHERE m.`movie`.`name` == 'The Godfather'
RETURN p.`person`.`name`;, Param: {}Error message: Query failed. Query: MATCH (p:`person`)-[:directed]->(m:`movie`) WHERE m.`movie`.`name` == 'The Godfather'
RETURN p.`person`.`name`;, Param: {}Error message: SemanticError: `person': Unknown tag
ERROR:llama_index.graph_stores.nebulagraph:Query failed. Query: MATCH (p:`pe

RetryError: RetryError[<Future at 0x7f8b728d2ad0 state=finished raised ValueError>]

In [97]:
graph_query = query_engine.generate_query(
    "math?",
)

display(Markdown(f"""
```cypher
{graph_query}
```
"""))


```cypher

MATCH (p:`person`)-[:directed]->(m:`movie`) WHERE m.`movie`.`name` == 'The Godfather'
RETURN p.`person`.`name`;
```


In [113]:
%%ngql 
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'math'
RETURN p.`entity`.`name`, e.relationship, m.`entity`.`name`;

INFO:nebula3.logger:Get connection to ('0.0.0.0', 9669)


,p.entity.name,e.relationship,m.entity.name
0,math,is,beautiful
1,math,is,enjoyable
2,math,is,remedy


In [114]:
%ng_draw

<class 'pyvis.network.Network'> |N|=0 |E|=0